In [1]:
import functions.hf_config as hf_config
from functions.adviser_tool import create_adviser_tool
from functions.loader import create_loader_tool
from functions.python_repl_tool import create_python_repl
from functions.utils import get_llm, get_prompt
from getpass import getpass

In [2]:
import os

In [3]:
LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="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"
LANGSMITH_PROJECT="ohw_llm"

In [4]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = LANGSMITH_API_KEY

In [5]:
hf_config.set_hf_token(getpass())

 ········


In [6]:
from functions.db_creation import create_db_examples

In [7]:
#chroma = create_db_examples()

In [8]:
advisor_tool = create_adviser_tool()

In [9]:
advisor_tool

StructuredTool(name='adviser_tool', description='Use this tool to find a suitable dataset and code example', args_schema=<class 'functions.adviser_tool.AdviserParams'>, func=<function description_reader at 0x7fdd9839e160>)

In [10]:
loader_tool = create_loader_tool()

In [11]:
loader_tool

StructuredTool(name='load_climate_data', description='A general use function for downloading datasets from various sources on the internet.', args_schema=<class 'functions.loader.ClimateDataParams'>, func=<function load_climate_data at 0x7fdd6b27c400>)

In [12]:
repl_tool = create_python_repl()

In [13]:
repl_tool

Tool(name='python_repl', description='\n        You are a Python REPL specialized for scientific data analysis.\n    \n        You receive:\n        - A file path to the downloaded dataset. This path comes from the load_climate_data tool. Do not use any other path.\n        - Optionally, an example analysis function from advisor Tool that may be relevant to the query.\n        \n        NOTE: do NOT download any new data from anywhere. You should directly access the data at the path that is given to you.\n        \n        Your job:\n        1. If an example function is provided and fits the user’s query, use it directly with the dataset. Make sure you load the data!\n        2. If no suitable example function is provided, write your own analysis code using standard scientific Python packages such as:\n           - xarray (for handling datasets)\n           - matplotlib (for plotting, with clean labels and colorbars)\n           - numpy (for computations)\n           - cartopy or geopa

In [14]:
tools = [
    advisor_tool,
    loader_tool,
    repl_tool
]

In [15]:
llm = get_llm()

In [16]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [17]:
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=get_prompt(),
)

/home/.pixi/envs/default/lib/python3.12/site-packages/pydantic/json_schema.py:2191: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='Available stores to read from.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [18]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [21]:
r = agent_executor.invoke({"input": "can you please plot avg sea surface temperature for 1 hour in a small region of the arctic. your choice of time"})

#r = agent_executor.invoke({"input": "can you please plot a variable of your choice from the indian ocean dataset. just do 1 hour worth sometime in 2020"})



> Entering new AgentExecutor chain...

Invoking: `load_climate_data` with `{'store': 'gs://weatherbench2/datasets/era5/1959-2023_01_10-6h-240x121_equiangular_with_poles_conservative.zarr', 'variable': 'sea_surface_temperature', 'lon_range': [-30, 10], 'lat_range': [70, 80], 'time_range': ['2010-01-01T00:00:00', '2010-01-01T00:00:00'], 'chunks': {'time': 1}}`




/home/jovyan/ohw25_proj_datadashboard_llm/final_notebooks/functions/loader.py:254: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(


Saving to temp/sea_surface_temperature_20250822_163613.nc (estimated size: 0.00 GB)
(<xarray.DataArray 'sea_surface_temperature' (time: 1, latitude: 7, longitude: 7)> Size: 196B
dask.array<transpose, shape=(1, 7, 7), dtype=float32, chunksize=(1, 7, 7), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 56B 70.5 72.0 73.5 75.0 76.5 78.0 79.5
  * time       (time) datetime64[ns, UTC] 8B 2010-01-01 00:00:00+00:00
  * longitude  (longitude) float64 56B 0.0 1.5 3.0 4.5 6.0 7.5 9.0
Attributes:
    long_name:   Sea surface temperature
    short_name:  sst
    units:       K, 'temp/sea_surface_temperature_20250822_163613.nc')

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse tool input: {'arguments': '{"__arg1":"import xarray as xr, matplotlib.pyplot as plt\\nimport cartopy.crs as ccrs\\nimport cartopy.feature as cfeature\\n\\n# Load the dataset from the given path (already loaded as a DataArray?)\\n# The load_climate_data returned a DataArray directly, but we need to capture it.\\n# In this environment, the result of load_climate_data is stored in a variable called \'data\'.\\n\\ndata = _  # placeholder, will be replaced by actual DataArray from previous step\\n\\n# Compute spatial average (mean) over latitude and longitude\\nspatial_mean = data.mean(dim=[\'latitude\',\'longitude\']).values.item()\\n\\n# Plot the SST field\\nfig = plt.figure(figsize=(6,5))\\nax = plt.axes(projection=ccrs.NorthPolarStereo())\\n# Set extent to region\\nax.set_extent([-30,